In [11]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn import metrics


from google.colab import drive
drive.mount('/content/drive')


file_path = '/content/drive/MyDrive/PCOS_Adaboost/PCOS_data.csv'

# Load the CSV file into a DataFrame
df = pd.read_csv(file_path)
print("DataFrame loaded successfully.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
DataFrame loaded successfully.


In [12]:
# Extract the label
label_pcos = df["PCOS (Y/N)"]

# Drop unnecessary columns
columns_to_drop = ["Sl. No", "Patient File No.", "PCOS (Y/N)", "Unnamed: 44", "II    beta-HCG(mIU/mL)", "AMH(ng/mL)", "Endometrium (mm)", "Avg. F size (R) (mm)", "Avg. F size (L) (mm)", "Follicle No. (R)", "Follicle No. (L)", "RBS(mg/dl)", "PRG(ng/mL)", "Vit D3 (ng/mL)", "PRL(ng/mL)", "AMH(ng/mL)", "TSH (mIU/L)", "FSH/LH", "LH(mIU/mL)", "FSH(mIU/mL)", "II    beta-HCG(mIU/mL)", "  I   beta-HCG(mIU/mL)", "Hb(g/dl)"]
df.drop(columns=columns_to_drop, axis=1, inplace=True)

In [13]:
# Fill missing values
df["Marraige Status (Yrs)"].fillna(df['Marraige Status (Yrs)'].median(), inplace=True)
df["Fast food (Y/N)"].fillna(1, inplace=True)

In [14]:
df.head(5)

,Age (yrs),Weight (Kg),Height(Cm),BMI,Blood Group,Pulse rate(bpm),RR (breaths/min),Cycle(R/I),Cycle length(days),Marraige Status (Yrs),...,Waist:Hip Ratio,Weight gain(Y/N),hair growth(Y/N),Skin darkening (Y/N),Hair loss(Y/N),Pimples(Y/N),Fast food (Y/N),Reg.Exercise(Y/N),BP _Systolic (mmHg),BP _Diastolic (mmHg)
0,28,44.6,152.0,19.3,15,78,22,2,5,7.0,...,0.83,0,0,0,0,0,1.0,0,110,80
1,36,65.0,161.5,24.9,15,74,20,2,5,11.0,...,0.84,0,0,0,0,0,0.0,0,120,70
2,33,68.8,165.0,25.3,11,72,18,2,5,10.0,...,0.90,0,0,0,1,1,1.0,0,120,80
3,37,65.0,148.0,29.7,13,72,20,2,5,4.0,...,0.86,0,0,0,0,0,0.0,0,120,70
4,25,52.0,161.0,20.1,11,72,18,2,5,1.0,...,0.81,0,0,0,1,0,0.0,0,120,80


# **Building the Model**

In [15]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# Import train_test_split function
from sklearn.model_selection import train_test_split

from sklearn import metrics


Split the dataset

In [16]:
# Split the dataset into features and target
X = df.values  # Features (all columns except the last)
y = label_pcos.values  # Target

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test


In [17]:
# Use cross-validation to evaluate model performance
abc = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=50, learning_rate=1)

# Train Adaboost Classifer
model = abc.fit(X_train, y_train)




/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Look at the accuracy

In [18]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 1, 10],
    'base_estimator__max_depth': [1, 2, 3]
}
grid_search = GridSearchCV(estimator=abc, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

In [19]:
# Train the best AdaBoost Classifier
best_model.fit(X_train, y_train)

# Predict the response for the test dataset
y_pred = best_model.predict(X_test)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [20]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.8282208588957055
